In [7]:
with open('debug_fb.html', encoding='utf-8') as fp:
    page_source = fp.read()

In [8]:
import re

In [9]:
PAGE_URL = 'https://www.facebook.com/groups/2621840064559532?sorting_setting=CHRONOLOGICAL'
LOGIN_URL = "https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&next=https%3A%2F%2Fwww.facebook.com%2Fgroups%2F2621840064559532%3Fsorting_setting%3DCHRONOLOGICAL"
MY_IMG_LINK = '67716435_2340044926083401_8053815337931505664_n.jpg'

POST_PATTERN = (
    '<a class=.*? href="(https://www.facebook.com/groups/2621840064559532/posts/\d+)/\?__cft__\[0\]=.*?" role="link" tabindex'
    + ".*?"
    'aria-haspopup="menu" aria-label="Actions for this post"'
    + "(.*?)"
    + '<title>Shared with Members of Buy Nothing Fremont, Newark and Union City, CA</title>'
)
IMG_PATTERN = '"(https://scontent.*?)"'
TXT_PATTERN = '<div dir="auto" style="text-align:.*?">(.*?)</div>'
POST_PATTERN = re.compile(POST_PATTERN)
IMG_PATTERN = re.compile(IMG_PATTERN)
TXT_PATTERN = re.compile(TXT_PATTERN)


In [10]:
all_posts = re.findall(POST_PATTERN, page_source)

In [11]:
len(all_posts)

0

In [13]:
pattern = '<a class=.*? href="#" role="link" tabindex'
re.findall(pattern, page_source)

['<a class="x1i10hfl x1qjc9v5 xjbqb8w xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x1q0g3np x87ps6o x1lku1pv x1a2a7pz x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq" href="/groups/2621840064559532/events/" role="link" tabindex="0"><div class="x6s0dn4 x1q0q8m5 x1qhh985 xu3j5b3 xcfux6l x26u7qi xm0m39n x13fuv20 x972fbf x9f619 x78zum5 x1q0g3np x1iyjqo2 xs83m0k x1qughib xat24cr x11i5rnm x1mh8g0r xdj266r xeuugli x18d9i69 x1sxyh0 xurb0ha xexx8yu x1n2onr6 x1ja2u2z x1gg8mnh"><div class="x78zum5 xdt5ytf xq8finb x1xmf6yo x1e56ztr x1n2onr6 xamitd3"><svg fill="currentColor" viewBox="0 0 20 20" width="1em" height="1em" class="x1lliihq x1k90msu x2h7rmj x1qfuztq x198g3q0 x1qx5ct2 xw4jnvo"><g fill-rule="evenodd" transform="translate(-446 -398)"><g fill-rule="nonzero" transform="tran

In [14]:
import argparse
from email.mime.text import MIMEText
import logging
from logging.handlers import TimedRotatingFileHandler
import multiprocessing
import os
from pathlib import Path
import re
import random
import subprocess
import signal
import smtplib
from sys import platform
import time
from urllib.parse import quote
import pandas as pd

from datetime import datetime
from datetime import time as datetime_time
import pytz
from enum import Enum
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException, NoSuchElementException
import yaml
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

if platform == "win32":
    from win10toast import ToastNotifier



In [15]:
EXISTING_POST_FILENAME = 'existing_posts_fb.txt'
NEW_POST_FILENAME = 'new_posts_fb.txt'
LOGIN_FILENAME = 'login_fb.txt'
SETTING_FILENAME = 'settings_fb.yaml'
EXECUTION_LOG_FILENAME = 'execution_fb.log'
SLEEPING_TIME = 60
DEBUG_FILENAME = 'debug_fb.html'
DEFAULT_EMAIL = 'drhsheng@gmail.com'
NIGHT_SLEEPING_TIME = 3600
MAX_NUM_EMAIL_PER_DAY = 200
EMAIL_COUNTER_KEY = 'EMAIL_COUNTER'
GECKODRIVER_LOG = 'geckodriver.log'

In [16]:
def load_firefox():
    # firefox_profile = webdriver.FirefoxProfile()
    # firefox_profile.set_preference("general.useragent.override", "whatever you want")
    firefox_options = webdriver.FirefoxOptions()
    if platform.startswith('linux'):
        firefox_options.add_argument("--headless")
    if os.path.exists('/Users/shengh4/miniconda3/envs/acap_dev/bin/firefox'):
        firefox_options.binary_location = '/Users/shengh4/miniconda3/envs/acap_dev/bin/firefox'
    # firefox_options.add_argument("start-maximized")
    # firefox_options.add_argument("disable-infobars")
    # firefox_options.add_argument("--disable-extensions")
    # firefox_options.add_argument('--no-sandbox')
    # firefox_options.add_argument('--disable-application-cache')
    # firefox_options.add_argument('--disable-gpu')
    # firefox_options.add_argument("--disable-dev-shm-usage")
    # firefox_options.add_argument("user-agent={user_agent}")
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.cache.disk.enable", False)
    firefox_profile.set_preference("browser.cache.memory.enable", False)
    firefox_profile.set_preference("browser.cache.offline.enable", False)
    firefox_profile.set_preference("network.http.use-cache", False)
    # browser = webdriver.Firefox(options=firefox_options)
    browser = webdriver.Firefox(firefox_profile=firefox_profile, options=firefox_options)
    return browser


def web_login(login_url, login_filename, browser=None):
    try:
        time.sleep(2)
        browser.get(login_url)
        time.sleep(1)
        email_field = browser.find_element(By.CSS_SELECTOR, "input[name='email'][type='text']")
        password_field = browser.find_element(By.CSS_SELECTOR, "input[name='pass'][type='password']")
        login_field = browser.find_element(By.CSS_SELECTOR, "button[name='login'][type='submit']")

        with open(login_filename, 'rt', encoding='utf-8') as fp:
            username = fp.readline().strip()
            password = fp.readline().strip()
        email_field.send_keys(username)
        password_field.send_keys(password)
        login_field.click()

        # wait for 2-step
        try:
            receive_code = browser.find_element(By.CSS_SELECTOR, "input[aria-label='Login code'][type='text']")
            print('receive_code: ', receive_code)
        except NoSuchElementException:
            # maybe the page has not been loaded yet.
            receive_code = True
        while receive_code is not None:
            print('receive_code: ', receive_code)
            print('Log in not authorized yet, wait for 5 seconds.')
            time.sleep(5)
            # <input type="text" class="inputtext" id="approvals_code" name="approvals_code" tabindex="1" autocomplete="off" placeholder="Login code" aria-label="Login code">
            try:
                receive_code = browser.find_element(By.CSS_SELECTOR, "input[aria-label='Login code'][type='text']")
            except NoSuchElementException:
                print('Log in authorized yet, proceed to posts.')
                break
    except Exception as e:
        raise Exception(f'Login failed with message: {e}')


In [19]:
PAGE_URL = 'https://www.facebook.com/groups/2621840064559532?sorting_setting=CHRONOLOGICAL'
LOGIN_URL = "https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&next=https%3A%2F%2Fwww.facebook.com%2Fgroups%2F2621840064559532%3Fsorting_setting%3DCHRONOLOGICAL"
MY_IMG_LINK = '67716435_2340044926083401_8053815337931505664_n.jpg'

page_url = PAGE_URL
login_url = LOGIN_URL
login_filename = 'login_fb.txt'

browser = load_firefox()
web_login(LOGIN_URL, login_filename, browser=browser)

receive_code:  <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="a2a6ea0f-d86b-4266-a98c-732b62373562")>
receive_code:  <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="a2a6ea0f-d86b-4266-a98c-732b62373562")>
Log in not authorized yet, wait for 5 seconds.
receive_code:  <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="a2a6ea0f-d86b-4266-a98c-732b62373562")>
Log in not authorized yet, wait for 5 seconds.
receive_code:  <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="a2a6ea0f-d86b-4266-a98c-732b62373562")>
Log in not authorized yet, wait for 5 seconds.
Log in authorized yet, proceed to posts.


In [68]:
def scroll_shim(passed_in_driver, element, offsets=None):
    scroll_script = (
        "var viewPortHeight = Math.max(document.documentElement.clientHeight, window.innerHeight || 0);" 
        + "var elementTop = arguments[0].getBoundingClientRect().top;"
        + "window.scrollBy(0, elementTop-(viewPortHeight/2));"
    )
    passed_in_driver.execute_script(scroll_script, element)
    if offsets:
        scroll_nav_out_of_way = f'window.scrollBy{offsets};'
        passed_in_driver.execute_script(scroll_nav_out_of_way)

In [87]:
browser.get(page_url)
num_rolling_times = 2
time.sleep(random.randint(1, 3))
# Robot robot = new Robot()
for _ in range(num_rolling_times):
    browser.execute_script("window.scrollTo({top: Math.round(document.body.scrollHeight), behavior: 'smooth'});")
    time.sleep(random.randint(1, 3))
post_elements = browser.find_elements(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm'][href='#'][role='link']")
for ele in post_elements:
    try:
        scroll_shim(browser, ele)
        time.sleep(random.random())
        action = ActionChains(browser)
        action.move_to_element(ele).perform()
        time.sleep(random.random())
        # ele.send_keys(Keys.CONTROL, Keys.ENTER)
    except Exception as e:
        print(e)
page_source = browser.page_source

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="8b8a61be-caf6-42ed-af83-4da2008838f6")> {'x': 534, 'y': 2422}
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="8b395d46-bcf8-4176-90b5-88f2db51dd77")> {'x': 534, 'y': 3588}
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="32c1ff32-503a-42ad-8e20-ecb71f5ae103")> {'x': 534, 'y': 4672}
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="655c2888-7ab9-4b31-a1d9-df6e7b354056")> {'x': 534, 'y': 5498}
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="6f922b56-7364-4f4f-ad07-ef3131b79e65")> {'x': 534, 'y': 6925}
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6330f684-c2d3-40ff-9476-0e211825f072", element="

In [90]:
pattern = '<a class="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm" href=".*?" role="link" tabindex'
re.findall(pattern, page_source)

['<a class="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm" href="https://www.facebook.com/groups/2621840064559532/posts/5699257210151120/?__cft__[0]=AZVuHq6_SOfbCwCkFClnMI3YeBeuGjFoSlERUg4gYhsDE4t5Zc5wDrZVsU1rTxs4A0S-sYAf97zSTxW3FzUR5d2ZFQMaKw_GXyjA9oE0jaH4BhzZ9qXHmRBu6azGYXaWpYCADDq-f7GCzZKD1cvPeq_Tp4DdFyXZ0F3UA9RF2QkgGlUHaFIF0LK4y6OqkVV7sPo&amp;__tn__=%2CO%2CP-R" role="link" tabindex',
 '<a class="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm" href="https://www.facebook.com/groups/2621840064559532/posts/5699231453487029/?__cft__[0]=AZVRpQZFBOGX4wvOBcyAK4bnVE1nism0WYD-sT8o6WXW2StjUwAWe1f52pCN8MHQU7CuVxptt4cH6LygbDpb2bpHa-3t8c8jnl95NdEETUIt-nwS7jb2Xh9o5Y8rLaXaXfuqwFif

In [86]:
ele = post_elements[1]
scroll_shim(browser, ele) #, offsets=(0, -50))
#time.sleep(random.random() / 0.3 + 0.01)
time.sleep(1)
print("".join(element.text for element in ele.find_elements(By.CSS_SELECTOR, 'span')))
action = ActionChains(browser)
action.move_to_element(ele.find_element(By.CSS_SELECTOR, 'span')).perform()

9m


In [89]:
with open('debug_fb.html', 'wt', encoding='utf-8') as fp:
    print(page_source, file=fp)

In [95]:
POST_PATTERN = (
    '<a class=.*? href="(https://www.facebook.com/groups/2621840064559532/posts/\d+)/\?__cft__\[0\]=.*?" role="link" tabindex'
    + ".*?"
    'aria-haspopup="menu" aria-label="Actions for this post"'
    + "(.*?)"
    + '<title>Shared with Members of Buy Nothing Fremont, Newark and Union City, CA</title>'
)
POST_PATTERN = re.compile(POST_PATTERN)
all_posts = re.findall(POST_PATTERN, page_source)
img_pattern = IMG_PATTERN
txt_pattern = TXT_PATTERN
new_results = []
html_txt = []
results = []
for post in all_posts:
    post_link = post[0]
    post_content = post[1]
    all_imgs = re.findall(img_pattern, post_content)
    if all_imgs:
        all_imgs = [img_link for img_link in all_imgs if not MY_IMG_LINK in img_link]
    else:
        all_imgs = []

    if post_link:
        post_id = post_link.split('/')[-1]
    else:
        if all_imgs:
            post_id = all_imgs[0]
        else:
            post_id = post_content[:100]
        post_link = PAGE_URL
    all_txts = re.findall(txt_pattern, post_content)
    if all_txts:
        all_txts = ' '.join(all_txts)
    else:
        all_txts = ''
    all_txts_lower = all_txts.lower()
    if all_txts_lower.find('gift') != -1 or all_txts_lower.find('iso') == -1:
        results.append((post_id, post_link, all_txts, all_imgs))

In [96]:
results

[('5699194030157438',
  'https://www.facebook.com/groups/2621840064559532/posts/5699194030157438',
  'I have a tree in my backyard full of nectarines / tangerines.. they are like cuties. Large intersection  - Mission and Driscoll. DM me for address and time Dmd you, thank you I am interested m! Would love to be considered. Can pick up tomorrow anytime',
  ['https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/319642132_10160422354608774_7834845586714446198_n.jpg?stp=dst-jpg_s600x600&amp;_nc_cat=104&amp;ccb=1-7&amp;_nc_sid=5cd70e&amp;_nc_ohc=QPw2x7n3X3QAX-7HzEo&amp;tn=IrOl51xOLwwV5yG3&amp;_nc_ht=scontent-sjc3-1.xx&amp;oh=00_AfBPRtMDPZzqYaX4yuuBQtiRDSc4HSLSMuAxCucNBrDXmg&amp;oe=639BB74C',
   'https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/319560463_10160422354703774_4349273574120594872_n.jpg?stp=dst-jpg_s600x600&amp;_nc_cat=105&amp;ccb=1-7&amp;_nc_sid=5cd70e&amp;_nc_ohc=MZ3BNMilzE0AX_2jfQM&amp;_nc_ht=scontent-sjc3-1.xx&amp;oh=00_AfDOY_vNQ0pC4qov3H8RRZdAtcz6ZjC32ZlwkdbafJG_dA&amp;oe=639